# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [12]:
import networkx
import urllib2
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [13]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: implement function
    
# intializing nodes and current line to the first line to be read
    nodes=[]                  
    curr_line=gfile.readline()
    
# While condition to stop after collecting all nodes, "next line" is used as a cue
    while curr_line !='\n':
        
# The first few lines which start with * are ignored and we move to the next line
        if curr_line[0]=='*':
            curr_line=gfile.readline()
        else:
            
# first 2 letters of line are appended into nodes list
            nodes.append(curr_line[0:2])
            curr_line=gfile.readline()
    return nodes

    pass

Next implement a function to read in the edges from the input file.

In [14]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    
#Initialize variables
    interactions=[]  
    edges=[]
    ele=[]

#code to split lines by semi-colon to collect raw interactions among edges
    for l in gfile.readlines():
        
        if l[1]==':':
            interactions.extend(l[2:].strip().split(';'))
            
        elif l[2]==':':
            interactions.extend(l[3:].strip().split(';'))
            
        else:
            break

# Split interations to get individual edges as tuples of form (a,b)
    for i in interactions:
        i_split=i.split(',')
        j=0
        while j<len(i_split):
            k=j+1
            while k<len(i_split):
                ele=[i_split[j]]+[i_split[k]]
                edges.append(tuple(ele))
                k=k+1
            j=j+1
    return edges

    pass

The following code should now correctly create the graph.

In [15]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [16]:
def Search(graph, root):
    """Runs depth-first search through a graph, starting at a given root. Neighboring
    nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
# intialize one list for getting explored nodes and another stack 
    explored=[root]
    stack=[root]
    i=0

#Check neighbors of root  
    while len(stack)!=0:
        N=graph.neighbors(stack[i])
        N.sort()
        counter=0
        
#Indicator to check if all neighbors have been explored
        for j in N:
            if j in explored:
                counter=counter+1
            else:
                break
            
#If all neighbors of node have been explored, pop the node from the stack        
        if counter==len(N):
            stack.pop()
            i=i-1     
        else:
            explored.append(N[counter])
            stack.append(N[counter])
            i=i+1
            
    return explored

    pass   

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [17]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [18]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    
# initialize variables  
    CC=[]
    CC_nodes=[]
# getting sorted list of nodes which will be used to ensure we follow alphabetical order for
# picking source
    nodes=graph.nodes()
    nodes.sort()
    root=nodes[0]
    i=0

# while loop to get all connected components
    while i<len(nodes):
    
# CC_nodes has an ungrouped list of nodes of connected components as a list
# CC_curr is the connected component found in the current iteration
        CC_curr= Search(graph,root)
        CC.append(CC_curr)
        CC_nodes.extend(CC_curr)

# Finding the source of next connected component
        for j in nodes:
            if j in CC_nodes:
                continue 
            else:
                root=j
                break

        i=i+len(CC_curr)
    return CC
    pass

We will check correctness of your code by verifying that your output list is identical to our solution.

In [19]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [20]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True
